In [1]:
import torch
torch.cuda.empty_cache()

import sys 

sys.path.append(r'/user/gr1/delphi/dghezzi/UNet')

from torch.utils.data import DataLoader
from UNet import UNet, TwoPatchDataset, train_unet_with_patches
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training su dispositivo: {device}")


Training su dispositivo: cuda


In [2]:
import os
import numpy as np

directory = '/user/gr1/delphi/dghezzi/UNet/UNet_dataset/4000_1000_160_180_npy_double_input'
val_labels_dir = os.path.join(directory, "labels", "val")

pos_weights_center = []
pos_weights_frame = []

for label_file in os.listdir(val_labels_dir):
    if label_file.endswith('.npy'):
        heatmap = np.load(os.path.join(val_labels_dir, label_file))
        h, w = heatmap.shape
        assert h == 800 and w == 800, "Le heatmap devono essere 800x800"

        # --- REGIONI ---
        # Centro 416x416
        start_y = (h - 416) // 2
        start_x = (w - 416) // 2
        end_y = start_y + 416
        end_x = start_x + 416

        center = heatmap[start_y:end_y, start_x:end_x]
        frame_mask = np.ones_like(heatmap, dtype=bool)
        frame_mask[start_y:end_y, start_x:end_x] = False
        frame = heatmap[frame_mask]

        # --- POS WEIGHT CENTRO ---
        pos_mass_center = np.sum(center)
        neg_mass_center = center.size - pos_mass_center
        pos_weight_center = neg_mass_center / (pos_mass_center + 1e-6)
        pos_weights_center.append(pos_weight_center)

        # --- POS WEIGHT CORNICE ---
        pos_mass_frame = np.sum(frame)
        neg_mass_frame = frame.size - pos_mass_frame
        pos_weight_frame = neg_mass_frame / (pos_mass_frame + 1e-6)
        pos_weights_frame.append(pos_weight_frame)

# --- STATISTICHE ---
pos_weights_center = np.array(pos_weights_center)
pos_weights_frame = np.array(pos_weights_frame)

mean_center = np.mean(pos_weights_center)
err_center = np.std(pos_weights_center) / np.sqrt(len(pos_weights_center))

mean_frame = np.mean(pos_weights_frame)
err_frame = np.std(pos_weights_frame) / np.sqrt(len(pos_weights_frame))

print(f"Pos weight CENTRO (416x416):  {mean_center:.4f} ± {err_center:.4f}  (n={len(pos_weights_center)})")
print(f"Pos weight CORNICE (resto):   {mean_frame:.4f} ± {err_frame:.4f}  (n={len(pos_weights_frame)})")



Pos weight CENTRO (416x416):  92.1424 ± 0.3570  (n=1000)
Pos weight CORNICE (resto):   1035.3766 ± 6.7930  (n=1000)


In [3]:
# Istanzia modello
model = UNet(in_channels=1, out_channels=1)  # canale singolo per heatmap

# Percorsi dati (assicurati che ci siano cartelle separate per train e val)
directory = '/user/gr1/delphi/dghezzi/UNet/UNet_dataset/22500_2500_160_180_npy'
train_image_dir = f'{directory}/images/train'
train_label_dir = f'{directory}/labels/train'
val_image_dir = f'{directory}/images/val'
val_label_dir = f'{directory}/labels/val'

# Dataset e DataLoader
train_dataset = TwoPatchDataset(image_dir=train_image_dir, label_dir=train_label_dir, transform=None)
val_dataset = TwoPatchDataset(image_dir=val_image_dir, label_dir=val_label_dir, transform=None)

batch = 16

train_loader = DataLoader(
    train_dataset,
    batch_size=batch,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch,
    shuffle=False,
    pin_memory=True,
    num_workers=4,
    drop_last=False
)

# Avvia il training
train_unet_with_patches(
    model,
    train_loader,
    val_loader,
    num_epochs=100,
    lr=1e-3,
    device=device,
    patience=7,
    binary_threshold=0.97
)

torch.cuda.empty_cache()
print("Training completato e cache GPU svuotata.")


Epoch 1/100 [Val]: 100%|██████████| 157/157 [05:04<00:00,  1.94s/it]


Epoch 1 | Train Loss: 1.558121 | Val Loss: 1.582175
F1 @2px: 0.0069 | F1 @4px: 0.0166 | F1 @6px: 0.0205 | Weighted F1: 0.0158
 ==> Nuovo modello salvato (val loss e F1 migliorati)



Epoch 2/100 [Val]: 100%|██████████| 157/157 [05:14<00:00,  2.00s/it]


Epoch 2 | Train Loss: 1.475975 | Val Loss: 1.587978
F1 @2px: 0.0071 | F1 @4px: 0.0166 | F1 @6px: 0.0197 | Weighted F1: 0.0156
 ==> Nessun miglioramento. patience: 1/7



Epoch 3/100 [Val]: 100%|██████████| 157/157 [05:16<00:00,  2.02s/it]


Epoch 3 | Train Loss: 1.452881 | Val Loss: 1.765162
F1 @2px: 0.0066 | F1 @4px: 0.0153 | F1 @6px: 0.0185 | Weighted F1: 0.0146
 ==> Nessun miglioramento. patience: 2/7



Epoch 4/100 [Train]:  48%|████▊     | 671/1406 [19:01<21:00,  1.71s/it]Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib64/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/user/gr1/delphi/dghezzi/.venv/lib64/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 51, in _pin_memory_loop
    do_one_step()
  File "/user/gr1/delphi/dghezzi/.venv/lib64/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib64/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/user/gr1/delphi/dghezzi/.venv/lib64/python3.9/site-packages/torch/multiprocessing/reductions.py", line 307, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib64/python3.9/multiprocessing/resour

KeyboardInterrupt: 